In [4]:
import csv
from shapely.wkt import loads
from pyproj import Transformer

def calculate_utm_zone(longitude):
    """
    Calculate the UTM zone for a given longitude.
    """
    return int((longitude + 180) // 6) + 1

# Input and output file paths
input_csv = "../obstacles/30b_buildings.csv"  # Replace with your input file
output_txt = "../obstacles/polygon_coordinates_converted_dhaka.txt"  # Output file for x, y coordinates (space-separated)

# Open the input CSV and read data
with open(input_csv, mode="r") as infile, open(output_txt, mode="w") as outfile:
    reader = csv.DictReader(infile)
    
    scene_id = 0
    polygon_id = 0  # To assign unique IDs to each polygon
    x_min = float('inf')
    x_max = float('-inf')
    y_min = float('inf')
    y_max = float('-inf')
    buffer = 50

    for row in reader:
        # Extract center latitude and longitude for the polygon
        center_lat = float(row["latitude"])
        center_lon = float(row["longitude"])
        
        # Dynamically calculate the UTM zone and construct the transformer
        utm_zone = calculate_utm_zone(center_lon)
        utm_epsg = f"326{utm_zone}" if center_lat >= 0 else f"327{utm_zone}"  # Northern or Southern Hemisphere
        transformer = Transformer.from_crs("EPSG:4326", f"EPSG:{utm_epsg}", always_xy=True)
        
        geometry = row["geometry"]  # Extract WKT geometry from 'geometry' column
        shape = loads(geometry)  # Convert WKT to Shapely geometry
        
        # Check the type of geometry
        if shape.geom_type == 'Polygon':
            # Process single polygon
            polygons = [shape]
        elif shape.geom_type == 'MultiPolygon':
            # Process each polygon in the MultiPolygon
            polygons = list(shape.geoms)
        else:
            # Skip unsupported geometry types
            continue
        
        # Process each polygon
        for polygon in polygons:
            
            # Iterate through the vertices of the polygon's exterior
            for vertex_index, coord in enumerate(polygon.exterior.coords[:-1]):
                lon, lat = coord  # Longitude and Latitude
                x, y = transformer.transform(lon, lat)  # Convert to x, y

                x_min = min(x_min,x)
                x_max = max(x_max,x)
                y_min = min(y_min,y)
                y_max = max(y_max,y)
                
                # Write space-separated values: Polygon_ID Vertex_Index X Y
                outfile.write(f"a {scene_id} {polygon_id} {vertex_index} {x:.6f} {y:.6f}\n")
            
            polygon_id += 1  # Increment the polygon ID for each new polygon

    outfile.write(f"b {x_min-buffer} {x_max+buffer} {y_min-buffer} {y_max+buffer}\n")

    print(polygon_id)

print(f"Polygon vertices converted and saved to {output_txt}")


3455651
Polygon vertices converted and saved to ../obstacles/polygon_coordinates_converted_dhaka.txt
